In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import joblib
import numpy as np

def calculate_kseb_bill(units):
    if units <= 40:
        return units * 2.5
    elif units <= 80:
        return 100 + (units - 40) * 3.5
    elif units <= 120:
        return 100 + 140 + (units - 80) * 4.6
    elif units <= 200:
        return 100 + 140 + 184 + (units - 120) * 6.6
    elif units <= 300:
        return 100 + 140 + 184 + 480 + (units - 200) * 7.5
    else:
        return 100 + 140 + 184 + 480 + 750 + (units - 300) * 7.9

# Read data from another CSV file
df = pd.read_csv('combined_data_with_sums.csv')

# Select numeric columns
df = df.dropna(axis=1)
numeric_cols = df.select_dtypes(include=['int', 'float']).columns

# Drop non-numeric columns
df = df.drop(columns=df.columns.difference(numeric_cols))

# Prepare the data for training
x = df.drop('Scaled Row Sums', axis=1).values
y = df['Scaled Row Sums'].values

scaler = MinMaxScaler()
features_to_scale = ['windSpeed', 'Home office [kW]', 'Microwave [kW]']
for feature in features_to_scale:
    feature_index = df.columns.get_loc(feature)
    x[:, feature_index] = scaler.fit_transform(x[:, feature_index].reshape(-1, 1)).flatten()

# Train a linear regression model
lnr = LinearRegression().fit(x, y)

# Save the trained model
joblib.dump(lnr, 'energy_model.pkl')

# Save the scaler object
joblib.dump(scaler, 'scaler.pkl')

# Get the scaling parameters used for training
min_value = scaler.data_min_
max_value = scaler.data_max_

# Load the trained model
loaded_model = joblib.load('energy_model.pkl')

# Load the scaler object
scaler = joblib.load('scaler.pkl')

# Get user input for energy in kilowatts (kW)
input_data = []
input_data = []
for feature in df.drop('Scaled Row Sums', axis=1).columns:
    while True:
        user_input = input(f"Enter the value for {feature}: ")
        try:
            value = float(user_input)
            break  # If input is valid, exit the while loop
        except ValueError:
            print("Invalid input! Please enter a valid number.")

    input_data.append(value)

# Scale the user input for specific features
for feature in features_to_scale:
    feature_index = df.columns.get_loc(feature)
    input_data[feature_index] = scaler.transform([[input_data[feature_index]]])[0][0]

# Make prediction using the loaded model
predicted_energy = loaded_model.predict([input_data])
pd=predicted_energy*60

print(f"Predicted energy consumption: {predicted_energy[0]} kWh")
predicted_energy = predicted_energy.reshape(-1, 1)

# Calculate KSEB bill
bill_amount = calculate_kseb_bill(pd[0])
print(f"KSEB Bill Amount: Rs. {bill_amount}")

Predicted energy consumption: 3.5881492171665657 kWh
KSEB Bill Amount: Rs. 1018.6671477249545
